In [4]:
%cd "~/pln-brca-xp/notebooks"
%load_ext autoreload
%autoreload 2

/home/xabush/pln-brca-xp/notebooks


In [77]:

from utils_emb import *
from utils import *
import matplotlib.pyplot as plt
from MKLpy import metrics
from MKLpy.preprocessing import normalization, rescale_01
from sklearn.preprocessing import MinMaxScaler

In [6]:
st_df = pd.read_csv("/var/www/datasets/embedding_vector_state_and_outcome.csv", index_col="patient_ID")
ge_df = pd.read_csv("/var/www/datasets/merged-combat15.csv.xz", index_col="patient_ID")

In [7]:
tamoxifen_studies_v2 = ["GSE12093",  "GSE1379", "GSE17705", "GSE9893"]
st_tamx_df = st_df[st_df["series_id"].isin(tamoxifen_studies_v2)]
ge_out_df = ge_df.join(st_df["posOutcome"])
ge_tamx_df = ge_df.loc[st_tamx_df.index,:]
ge_out_tamx_df = ge_tamx_df.join(st_df["posOutcome"])

In [64]:
scaler = MinMaxScaler()
X_ge, y_ge = ge_out_df.drop(["posOutcome"], axis=1), ge_out_df["posOutcome"]
X_ge_tr, X_ge_te, y_ge_tr, y_ge_te = train_test_split(X_ge, y_ge, shuffle=True, test_size=0.3, stratify=y_ge)
X_ge_tx_tr, X_ge_tx_te, y_ge_tx_tr, y_ge_tx_te = get_train_test_set(ge_out_tamx_df, "/var/www/datasets/tamox_train_set.txt", "/var/www/datasets/tamox_test_set.txt")
X_ge_tr_sc, X_ge_te_sc = pd.DataFrame(scaler.fit_transform(X_ge_tr), index=X_ge_tr.index), pd.DataFrame(scaler.transform(X_ge_te), index=X_ge_te.index)
scaler = MinMaxScaler()
X_ge_tx_tr_sc, X_ge_tx_te_sc = pd.DataFrame(scaler.fit_transform(X_ge_tx_tr), index=X_ge_tx_tr.index), pd.DataFrame(scaler.transform(X_ge_tx_te), index=X_ge_tx_te.index)

In [9]:
feats_100_mrmr = load_features("/var/www/datasets/feats_100_raw_nn.txt")
ge_diff_100_df = pd.read_csv("https://raw.githubusercontent.com/singnet/cancer/master/data/curatedBreastData/diffExp/top100genes4studies.csv")
feats_diff_100 = ge_diff_100_df["symbol"].tolist()

In [10]:
qnorm = MQNormalizer()
X_ge_tr_q = qnorm.fit_transform(X_ge_tr)
X_ge_te_q = qnorm.transform(X_ge_te)

In [12]:
qnorm_2 = MQNormalizer()
X_ge_tx_tr_q = qnorm_2.fit_transform(X_ge_tx_tr)
X_ge_tx_te_q = qnorm_2.transform(X_ge_tx_te)

In [15]:
feats_100_q = load_features("/var/www/datasets/feats_100_mrmr_tamoxifen_qnormalized.txt")
X_ge_tx_tr_q_100, X_ge_tx_te_q_100 = X_ge_tx_tr_q[feats_100_q], X_ge_tx_te_q[feats_100_q]

In [16]:
X_ge_tx_mrmr_tr, X_ge_tx_mrmr_te = X_ge_tx_tr_sc[feats_100_mrmr], X_ge_tx_te_sc[feats_100_mrmr]
X_ge_tx_diff_tr, X_ge_tx_diff_te = X_ge_tx_tr_sc[feats_diff_100], X_ge_tx_te_sc[feats_diff_100]

In [17]:
X_ppt_mrmr_tr, X_ppt_mrmr_te = pd.read_table("/var/www/datasets/embedding-vectors/tamoxifen_group/property_vector_100genes_ge_pln_TRAIN_2021-03-25.csv", index_col="patient_ID"), pd.read_table("/var/www/datasets/embedding-vectors/tamoxifen_group/property_vector_100genes_ge_pln_TEST_2012-03-25.csv", index_col="patient_ID")
print(X_ppt_mrmr_tr.shape)
y_ppt_mrmr_tr, y_ppt_mrmr_te = X_ppt_mrmr_tr.join(st_tamx_df["posOutcome"])["posOutcome"], X_ppt_mrmr_te.join(st_tamx_df["posOutcome"])["posOutcome"]

(449, 6748)


In [20]:
X_ppt_diff_tr, X_ppt_diff_te = pd.read_table("/var/www/datasets/embedding-vectors/tamoxifen_group/diffexp/property_vector_Train_2021-04-02.csv", index_col="patient_ID"), pd.read_table("/var/www/datasets/embedding-vectors/tamoxifen_group/diffexp/property_vector_Test_2021-04-02.csv", index_col="patient_ID")
print(X_ppt_diff_tr.shape)
y_ppt_diff_tr, y_ppt_diff_te = X_ppt_diff_tr.join(st_tamx_df["posOutcome"])["posOutcome"], X_ppt_diff_te.join(st_tamx_df["posOutcome"])["posOutcome"]

(449, 3740)


In [65]:
kernel_dicts = {"tanimoto": tanimoto, "tanimoto_v2":tanimoto_v2, "intsn": intensional_similarity, "rbf": "rbf", "cosine": "cosine", "linear": "linear", "poly": "poly"}
comp_dict = {"cross": [X_ge_tr_sc, y_ge_tr], "cross_qnorm": [X_ge_tr_q, y_ge_tr], "tx": [X_ge_tx_tr_sc, y_ge_tx_tr],
             "tx_qnorm" : [X_ge_tx_tr_q, y_ge_tx_tr] ,"ft_100_mrmr": [X_ge_tx_mrmr_tr, y_ge_tx_tr],
             "ft_100_diff": [X_ge_tx_diff_tr, y_ge_tx_tr], "ft_100_qnorm": [X_ge_tx_tr_q_100 ,y_ge_tx_tr],
             "ppt_mrmr": [X_ppt_mrmr_tr, y_ppt_mrmr_tr], "ppt_diff": [X_ppt_diff_tr, y_ppt_diff_tr]}
res_dict = {}

In [66]:
for j in comp_dict:
    X, y = comp_dict[j][0], comp_dict[j][1]
    X, y = X.values, y.values
    scores, _ = compute_ker_alignment(X, X, y, kernel_dicts, {"rbf": {"gamma": 1}})
    res_dict[j] = scores

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   2 out of   7 | elapsed:    3.7s remaining:    9.2s
[Parallel(n_jobs=-1)]: Done   3 out of   7 | elapsed:    3.7s remaining:    4.9s
[Parallel(n_jobs=-1)]: Done   4 out of   7 | elapsed:    3.8s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done   5 out of   7 | elapsed:  1.8min remaining:   43.9s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:  4.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:  4.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done   2 out of   7 | elapsed:    4.3s remaining:   10.8s
[Parallel(n_jobs=-1)]: Done   3 out of   7 | elapsed:    4.4s remaining:    5.9s
[Parallel(n_jobs=-1)]: Done   4 out of   7 | elapsed:    4.6s remaining:    3.5s
[Pa

In [67]:
tmp_dict = {}
for k in res_dict:
    tmp_dict[k] = {}
    for i, j in zip(res_dict[k], kernel_dicts):
        tmp_dict[k][j] = i[j]

In [68]:
res_df = pd.DataFrame(tmp_dict)
display(res_df)

,cross,cross_qnorm,tx,tx_qnorm,ft_100_mrmr,ft_100_diff,ft_100_qnorm,ppt_mrmr,ppt_diff
tanimoto,0.014594,0.009048,0.038004,0.031547,0.138521,0.084550,0.093139,0.073242,0.057340
tanimoto_v2,0.033952,0.009625,0.048026,0.033113,0.137042,0.084362,0.088042,0.079524,0.059477
intsn,0.033952,0.009625,0.048026,0.033113,0.137042,0.084364,0.088042,0.079524,0.059477
rbf,0.025250,0.025211,0.037154,0.037009,0.037367,0.037291,0.037313,0.037378,0.056661
cosine,0.014039,0.006456,0.036814,0.025486,0.139528,0.085365,0.075495,0.069036,0.055554
linear,0.013152,0.006493,0.037112,0.027790,0.139491,0.085944,0.115904,0.083570,0.057686
poly,0.013001,0.007131,0.037250,0.030165,0.134488,0.083608,0.128964,0.083629,0.057689


In [78]:
s, k = compute_ker_alignment(X_ge_tx_mrmr_tr.values, X_ge_tx_mrmr_tr.values, y_ge_tx_tr.values, {"rbf": "rbf"})

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.5s finished


In [79]:
k

([tensor([[ 0.8104,  0.0154,  0.0746,  ...,  0.0593, -0.0891, -0.0320],
          [ 0.0154,  0.7939, -0.0335,  ..., -0.0640, -0.0464, -0.0703],
          [ 0.0746, -0.0335,  0.6306,  ...,  0.0657,  0.0253,  0.1029],
          ...,
          [ 0.0593, -0.0640,  0.0657,  ...,  0.6915,  0.0152,  0.0688],
          [-0.0891, -0.0464,  0.0253,  ...,  0.0152,  0.5851,  0.0018],
          [-0.0320, -0.0703,  0.1029,  ...,  0.0688,  0.0018,  0.4750]],
         dtype=torch.float64)],)